In [1]:
import os
import yaml
from geopandas import GeoDataFrame
import pytest
from network_wrangler import ProjectCard
import numpy as np
import pandas as pd

## remote i/o

In [2]:
input_dir = "."
card_dir = os.path.join(input_dir, 'reference_projects')

filename_registry = os.path.join(input_dir, 'registry.csv')
filename_output_registry = os.path.join(input_dir, 'test_registry_update.csv')
filename_config = os.path.join(input_dir, 'registry_config.yml')

## data reads

In [3]:
card_filenames = []
for (dirpath, dirnames, filenames) in os.walk(card_dir):
    for filename in filenames:
        name, extension = os.path.splitext(filename)
        if (extension in ['.yml', '.yaml']):
            card_filenames.append(os.path.join(card_dir, filename))
    break

card_filenames

['./reference_projects/project_B.yml', './reference_projects/project_A.yml']

In [4]:
registry_df = pd.read_csv(filename_registry)
with open(filename_config, "r") as config_file:
    config_dict = yaml.safe_load(config_file)

config_dict

{'start_node_number': 1000, 'start_link_number': 500}

## update the registry

In [5]:
node_df = registry_df[registry_df['node_or_link'] == 'node']
link_df = registry_df[registry_df['node_or_link'] == 'link']

In [6]:
card_index = 0
for card_filename in card_filenames:
    write_updated_card = False
    card = ProjectCard.read(card_filename, validate = False)
    card_dict = card.__dict__

    if (card_dict['category'] == 'New Roadway'):
    
        node_index = 0
        for node in card_dict['nodes']:
            new_node = node['model_node_id']
        
            if (new_node <= config_dict['start_node_number']):
                msg = "New node number ({}) in project '{}' is less than the starting node number in config file of {}".format(
                        new_node, 
                        card_dict['project'], 
                        config_dict['start_node_number'],
                    )
                raise ValueError(msg)
            
            if (new_node not in node_df['number'].values):
                df = pd.DataFrame( 
                    {'node_or_link': 'node',
                     'number': [new_node], 
                     'project': [card_dict['project']],
                    } 
                ) 
                node_df = node_df.append(df)
            else:
                number = 1 + node_df['number'].max()
                card_dict['nodes'][node_index]['model_node_id'] = number
                for i in range(0, len(card_dict['links'])):
                    if (card_dict['links'][i]['A'] == new_node):
                        card_dict['links'][i]['A'] = number
                    if (card_dict['links'][i]['B'] == new_node):
                        card_dict['links'][i]['B'] = number
                df = pd.DataFrame( 
                    {'node_or_link': 'node',
                     'number': [number], 
                     'project': [card_dict['project']] } ) 
                node_df = node_df.append(df)
                write_updated_card = True
                
            node_index = node_index + 1
            
        link_index = 0
        for link in card_dict['links']:
            new_link = link['model_link_id']
        
            if (new_link <= config_dict['start_link_number']):
                msg = "New link id ({}) in project '{}' is less than the starting link number in config file of {}".format(
                        new_link, 
                        card_dict['project'], 
                        config_dict['start_link_number'],
                    )
                raise ValueError(msg)
            
            if (new_link not in link_df['number'].values):
                df = pd.DataFrame(
                    {'node_or_link': 'link',
                     'number': [new_link], 
                     'project':[card_dict['project']]
                    }
                ) 
                link_df = link_df.append(df)
            else:
                number = 1 + link_df['number'].max()
                card_dict['links'][link_index]['model_link_id'] = number
                for i in range(0, len(card_dict['links'])):
                    if (card_dict['links'][i]['model_link_id'] == new_link):
                        card_dict['links'][i]['model_link_id'] = number
                df = pd.DataFrame(
                    {'node_or_link': 'link',
                     'number': [number],
                     'project': [card_dict['project']]
                    }
                ) 
                link_df = link_df.append(df)
                write_updated_card = True
                
            link_index = link_index + 1

    if (write_updated_card):
        card.__dict__.update(card_dict)
        name, extension = os.path.splitext(card_filename)
        card.write(filename = (name + '_updated' + extension))

2021-03-26 19:19:19, INFO: Wrote project card to: ./reference_projects/project_A_updated.yml


In [7]:
out_df = node_df.append(link_df)

In [8]:
out_df

,node_or_link,number,project
0,node,1001,Project B
0,node,1002,Project B
0,node,1003,Project A
0,node,1004,Project A
0,link,501,Project B
0,link,502,Project A


## write to disk

In [10]:
out_df.to_csv(filename_output_registry)